In [ ]:
!pip install medmnist

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
import pandas as pd
from torchvision import datasets, models, transforms
import torch.utils.data as data
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import Dataset
from PIL import Image
from medmnist import INFO, Evaluator
import medmnist

In [ ]:
data_flag = 'octmnist'
download = True
info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

In [ ]:
DataClass=getattr(medmnist, info['python_class'])

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
feature_extract = False #when True we only update the reshaped layer params
num_epochs = 5
batch_size = 32
num_classes = n_classes
input_size=299 #DO NOT CHANGE

In [ ]:
data_transform=""
if n_channels==1:
  data_transform = transforms.Compose([
    transforms.Resize(input_size),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)) ,
    transforms.Normalize(mean=[.5], std=[.5])
])
else:
  data_transform = transforms.Compose([
    transforms.Resize(input_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])



In [ ]:
train_dataset = DataClass(split='train', transform=data_transform, download=download)
test_dataset = DataClass(split='test', transform=data_transform, download=download)

  0%|          | 0/54938180 [00:00<?, ?it/s]

Using downloaded and verified file: /root/.medmnist/octmnist.npz


In [ ]:
train_dataset_loader = data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False)
test_dataset_loader = data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model = models.inception_v3(pretrained=True)
set_parameter_requires_grad(model, feature_extract)
# Handle the auxilary net
num_ftrs = model.AuxLogits.fc.in_features
model.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
# Handle the primary net
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs,num_classes)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
model = model.to(device)
params_to_update = model.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

Params to learn:
	 Conv2d_1a_3x3.conv.weight
	 Conv2d_1a_3x3.bn.weight
	 Conv2d_1a_3x3.bn.bias
	 Conv2d_2a_3x3.conv.weight
	 Conv2d_2a_3x3.bn.weight
	 Conv2d_2a_3x3.bn.bias
	 Conv2d_2b_3x3.conv.weight
	 Conv2d_2b_3x3.bn.weight
	 Conv2d_2b_3x3.bn.bias
	 Conv2d_3b_1x1.conv.weight
	 Conv2d_3b_1x1.bn.weight
	 Conv2d_3b_1x1.bn.bias
	 Conv2d_4a_3x3.conv.weight
	 Conv2d_4a_3x3.bn.weight
	 Conv2d_4a_3x3.bn.bias
	 Mixed_5b.branch1x1.conv.weight
	 Mixed_5b.branch1x1.bn.weight
	 Mixed_5b.branch1x1.bn.bias
	 Mixed_5b.branch5x5_1.conv.weight
	 Mixed_5b.branch5x5_1.bn.weight
	 Mixed_5b.branch5x5_1.bn.bias
	 Mixed_5b.branch5x5_2.conv.weight
	 Mixed_5b.branch5x5_2.bn.weight
	 Mixed_5b.branch5x5_2.bn.bias
	 Mixed_5b.branch3x3dbl_1.conv.weight
	 Mixed_5b.branch3x3dbl_1.bn.weight
	 Mixed_5b.branch3x3dbl_1.bn.bias
	 Mixed_5b.branch3x3dbl_2.conv.weight
	 Mixed_5b.branch3x3dbl_2.bn.weight
	 Mixed_5b.branch3x3dbl_2.bn.bias
	 Mixed_5b.branch3x3dbl_3.conv.weight
	 Mixed_5b.branch3x3dbl_3.bn.weight
	 Mixed_5b.b

In [ ]:
optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
criterion=nn.CrossEntropyLoss()

In [ ]:
for epoch in range(num_epochs):
  running_loss = 0.0
  running_corrects = 0
  for inputs, labels in train_dataset_loader:
    inputs = inputs.to(device)
    labels = labels.flatten()
    labels=labels.to(device)
    optimizer.zero_grad()
    outputs, aux_outputs = model(inputs)
    loss1 = criterion(outputs, labels)
    loss2 = criterion(aux_outputs, labels)
    loss = loss1 + 0.4*loss2

    _, preds = torch.max(outputs, 1)

    loss.backward()
    optimizer.step()

    running_loss += loss.item() * inputs.size(0)
    running_corrects += torch.sum(preds == labels.data)

  epoch_loss = running_loss / len(train_dataset_loader.dataset)
  epoch_acc = running_corrects.double() / len(train_dataset_loader.dataset)
  print(str(epoch)+" Loss: "+str(epoch_loss)+" Acc: "+str(epoch_acc))

0 Loss: 0.49347900015566454 Acc: tensor(0.8788, device='cuda:0', dtype=torch.float64)
1 Loss: 0.30396663799464757 Acc: tensor(0.9266, device='cuda:0', dtype=torch.float64)
2 Loss: 0.2146019054439574 Acc: tensor(0.9492, device='cuda:0', dtype=torch.float64)
3 Loss: 0.15380219683682914 Acc: tensor(0.9643, device='cuda:0', dtype=torch.float64)
4 Loss: 0.12152968022393795 Acc: tensor(0.9709, device='cuda:0', dtype=torch.float64)


In [ ]:
predictions=[]
true_labels=[]
for inputs, labels in test_dataset_loader:
  inputs=inputs.to(device)
  labels = labels.flatten()
  outputs,_=model(inputs)
  predictions.extend(torch.max(outputs, 1)[1].cpu())
  true_labels.extend(labels)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(true_labels, predictions))

              precision    recall  f1-score   support

           0       0.60      0.97      0.74       250
           1       0.96      0.73      0.83       250
           2       0.94      0.26      0.41       250
           3       0.72      0.98      0.83       250

    accuracy                           0.73      1000
   macro avg       0.81      0.73      0.70      1000
weighted avg       0.81      0.73      0.70      1000

